In [4]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from imblearn.over_sampling import SMOTE
import mlflow
import mlflow.sklearn

import optuna
import dagshub
import pickle

dagshub.init(repo_owner='satyajeetrai007', repo_name='Youtube-Comment-Sentiment-Analysis', mlflow=True)
mlflow.set_experiment("Algorithms with HyperParameter tuning")

df = pd.read_csv('data_preprocessed.csv').dropna().drop_duplicates()
df.shape

Initialized MLflow to track repo "satyajeetrai007/Youtube-Comment-Sentiment-Analysis"

Repository satyajeetrai007/Youtube-Comment-Sentiment-Analysis initialized!

(36243, 2)

In [5]:

# Step 1: Clean and isolate features and target
df = df.dropna(subset=['category'])
X = df['clean_comment']
y = df['category']

# Step 2: Perform the train-test split FIRST to prevent data leakage 🧪
# 'stratify=y' ensures the class distribution is preserved in both splits.
X_train_text, X_test_text, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Step 3: Fit the TF-IDF vectorizer ONLY on the training data
vectorizer = TfidfVectorizer(ngram_range=(1, 3), max_features=2000)
X_train_vec = vectorizer.fit_transform(X_train_text)

# Use the same fitted vectorizer to transform the test data
X_test_vec = vectorizer.transform(X_test_text)

# Step 4: Apply SMOTE to the TRAINING data ONLY to balance it
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_vec, y_train)

# The test data (X_test_vec, y_test) remains in its original, imbalanced state for a realistic evaluation.

# --- MLflow and Optuna Functions (no changes needed) ---

def log_mlflow(model_name, model, X_train, X_test, y_train, y_test):
    with mlflow.start_run():
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams_Corrected")
        mlflow.set_tag("experiment_type", "algorithm_comparison")
        mlflow.log_param("algo_name", model_name)

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        model_filename = f"{model_name}_model.pkl"
        with open(model_filename, "wb") as f:
            pickle.dump(model, f)
        mlflow.log_artifact(model_filename, "model")

# Step 5: Optuna objective function for Multinomial Naive Bayes
def objective_mnb(trial):
    alpha = trial.suggest_float('alpha', 1e-4, 1.0, log=True)
    
    model = MultinomialNB(alpha=alpha)
    # Train on the resampled training data
    model.fit(X_train_resampled, y_train_resampled)
    
    # Evaluate on the original, untouched test data
    y_pred = model.predict(X_test_vec)
    return accuracy_score(y_test, y_pred)

# Step 6: Run Optuna for Multinomial Naive Bayes
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_mnb, n_trials=30)

    best_params = study.best_params
    best_model = MultinomialNB(alpha=best_params['alpha'])
    
    # Log the best performing model with MLflow
    log_mlflow("MultinomialNB", best_model, X_train_resampled, X_test_vec, y_train_resampled, y_test)

# Run the experiment
run_optuna_experiment()

[I 2025-08-27 20:21:43,124] A new study created in memory with name: no-name-c7d10bb7-5b31-4baf-90b8-7a53fe98d46e
[I 2025-08-27 20:21:43,194] Trial 0 finished with value: 0.691129811008415 and parameters: {'alpha': 0.0021271211084827506}. Best is trial 0 with value: 0.691129811008415.
[I 2025-08-27 20:21:43,212] Trial 1 finished with value: 0.6897503103876397 and parameters: {'alpha': 0.017909867279413672}. Best is trial 0 with value: 0.691129811008415.
[I 2025-08-27 20:21:43,234] Trial 2 finished with value: 0.6846461580907711 and parameters: {'alpha': 0.34529574056364637}. Best is trial 0 with value: 0.691129811008415.
[I 2025-08-27 20:21:43,252] Trial 3 finished with value: 0.69140571113257 and parameters: {'alpha': 0.00021485228542195272}. Best is trial 3 with value: 0.69140571113257.
[I 2025-08-27 20:21:43,271] Trial 4 finished with value: 0.6897503103876397 and parameters: {'alpha': 0.014883348493884278}. Best is trial 3 with value: 0.69140571113257.
[I 2025-08-27 20:21:43,291] T

🏃 View run MultinomialNB_SMOTE_TFIDF_Trigrams_Corrected at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/6/runs/a08923686aac42b8afc2bec834311193
🧪 View experiment at: https://dagshub.com/satyajeetrai007/Youtube-Comment-Sentiment-Analysis.mlflow/#/experiments/6
